In [1]:
import matplotlib.pyplot as plt
from PIL import Image
import cv2
import numpy as np
import os
from ultralytics import YOLO
import chess
from stockfish import Stockfish
import time
import threading

In [2]:
model_path = os.path.join('.', 'runs', 'segment', 'train_CB', 'weights', 'best.pt')
model = YOLO(model_path)

model_path1 = os.path.join('.', 'runs', 'detect', 'train_GCP', 'weights', 'best.pt')
model1 = YOLO(model_path1)

video = cv2.VideoCapture(0)
address = "https://192.168.1.5:8080/video" #ADD CUSTOM ADDRESS HERE!
video.open(address)

if not video.isOpened():
    print("Error: Could not open video stream.")
    exit()

stockfish=Stockfish("StockFish/stockfish-windows-x86-64-avx2.exe")

top_moves = []

while True:
    ret, frame = video.read()
    if not ret:
        print("Error: Failed to capture frame.")
        break

    results = model.predict(frame, conf=0.5)

    warping_done = False

    for result in results:
        
        if hasattr(result, 'masks') and result.masks is not None:
            masks = result.masks.data.cpu().numpy()  

            for i in range(masks.shape[0]):
                
                mask = masks[i]
                mask_binary = (mask * 255).astype(np.uint8)
                mask_binary_resized = cv2.resize(mask_binary, (frame.shape[1], frame.shape[0]), interpolation=cv2.INTER_NEAREST)
                masked_frame = cv2.bitwise_and(frame, frame, mask=mask_binary_resized)
                contours, _ = cv2.findContours(mask_binary_resized, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
                
                if contours:
                    
                    largest_contour = max(contours, key=cv2.contourArea)
                    epsilon = 0.02 * cv2.arcLength(largest_contour, True)
                    approx_corners = cv2.approxPolyDP(largest_contour, epsilon, True)

                    if len(approx_corners) == 4:
                        
                        src_pts = np.array([pt[0] for pt in approx_corners], dtype="float32")
                        dst_pts = np.array([[0, 0], [600, 0], [600, 600], [0, 600]], dtype="float32")
                        
                        M = cv2.getPerspectiveTransform(src_pts, dst_pts)
                        warped_frame = cv2.warpPerspective(masked_frame, M, (600, 600))
                        rotated_frame = cv2.rotate(warped_frame, cv2.ROTATE_90_CLOCKWISE)
                        mirrored_frame = cv2.flip(rotated_frame, 1)

                        grid_size = 600
                        rows, cols = 8, 8
                        cell_width = grid_size // cols
                        cell_height = grid_size // rows

                        img_with_grid = mirrored_frame.copy()
                        for r in range(1, rows):
                            y = r * cell_height
                            cv2.line(img_with_grid, (0, y), (grid_size, y), (255, 0, 0), 1)
                        for c in range(1, cols):
                            x = c * cell_width
                            cv2.line(img_with_grid, (x, 0), (x, grid_size), (255, 0, 0), 1)

                        warping_done = True
                    else:
                        print("Could not find 4 corners for perspective transformation.")
                else:
                    print("No contours found for the mask.")
        else:
            print("No masks found in the results.")
    
    if not warping_done:
        for result in results:
            if hasattr(result, 'boxes') and result.boxes is not None:
                boxes = result.boxes.xyxy.cpu().numpy() 
                
                if len(boxes) > 0:
                    print("in")
                    x_min, y_min, x_max, y_max = boxes[0]
                    src_pts = np.array([[x_min, y_min], [x_max, y_min], [x_max, y_max], [x_min, y_max]], dtype="float32")
                    dst_pts = np.array([[0, 0], [600, 0], [600, 600], [0, 600]], dtype="float32")

                    M = cv2.getPerspectiveTransform(src_pts, dst_pts)
                    warped_frame = cv2.warpPerspective(frame, M, (600, 600))
                    rotated_frame = cv2.rotate(warped_frame, cv2.ROTATE_90_CLOCKWISE)
                    mirrored_frame = cv2.flip(rotated_frame, 1)

                    grid_size = 600
                    rows, cols = 8, 8
                    cell_width = grid_size // cols
                    cell_height = grid_size // rows
                    
                    img_with_grid = mirrored_frame.copy()
                    print(img_with_grid)
                    for r in range(1, rows):
                        y = r * cell_height
                        cv2.line(img_with_grid, (0, y), (grid_size, y), (255, 0, 0), 1)
                    for c in range(1, cols):
                        x = c * cell_width
                        cv2.line(img_with_grid, (x, 0), (x, grid_size), (255, 0, 0), 1)
                        
                    warping_done = True
                    break

    if not warping_done:
        frameResized = cv2.resize(frame, (600, 600))
        cv2.imshow("Chessboard", frameResized)
            
    else:
        columns = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h']
        rows_labels = ['1', '2', '3', '4', '5', '6', '7', '8']
        rows_labels.reverse()
    
        img_with_labels = img_with_grid.copy()
    
        for i in range(rows):
            for j in range(cols):
                x = i * cell_width
                y = j * cell_height
        
                label = columns[i] + rows_labels[j]
    
                text_x = x + cell_width // 2 - 15 
                text_y = y + cell_height // 2 + 15
        
                cv2.putText(img_with_labels, label, (text_x, text_y), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
    
        res = img_with_labels.copy()
    
        piece_positions = {}
        
        results1 = model1.predict(mirrored_frame, conf=0.7)
        
        for result in results1:
            
            boxes = result.boxes.xyxy.cpu().numpy()
            classes = result.boxes.cls.cpu().numpy()
            names = result.names
            
            for box,class_id in zip(boxes,classes):
                
                x_min,y_min,x_max,y_max = box
                center_x = (x_min + x_max) / 2
                center_y = ((y_min + y_max) / 2)
                
                grid_col = int(center_x // cell_width)
                grid_row = int(center_y // cell_height)
                
                if 0 <= grid_col < cols and 0 <= grid_row < rows:
                    
                    grid_label = columns[grid_col] + rows_labels[grid_row]
                    piece_positions[grid_label] = names[int(class_id)]
                    
                    cv2.circle(res, (int(center_x), int(center_y)), 5, (255, 0, 0), -1)
                    cv2.putText(res, grid_label, (int(center_x)-10, int(center_y)-15), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
    
        piece_map = {
            'white-rook': chess.Piece(chess.ROOK, chess.WHITE),
            'white-knight': chess.Piece(chess.KNIGHT, chess.WHITE),
            'white-bishop': chess.Piece(chess.BISHOP, chess.WHITE),
            'white-queen': chess.Piece(chess.QUEEN, chess.WHITE),
            'white-king': chess.Piece(chess.KING, chess.WHITE),
            'white-pawn': chess.Piece(chess.PAWN, chess.WHITE),
            'black-rook': chess.Piece(chess.ROOK, chess.BLACK),
            'black-knight': chess.Piece(chess.KNIGHT, chess.BLACK),
            'black-bishop': chess.Piece(chess.BISHOP, chess.BLACK),
            'black-queen': chess.Piece(chess.QUEEN, chess.BLACK),
            'black-king': chess.Piece(chess.KING, chess.BLACK),
            'black-pawn': chess.Piece(chess.PAWN, chess.BLACK),
        }
        
        def set_up_board(board, piece_positions, top_moves):
    
            try:
                for position, piece_description in piece_positions.items():
                    piece = piece_map.get(piece_description)
                    if piece:
                        square = chess.parse_square(position)
                        board.set_piece_at(square, piece)
                    else:
                        raise ValueError(f"Invalid piece description: {piece_description}")
            except Exception as e:
                print(f"Error setting up board: {e}")
                return top_moves
        
            fen_string = board.fen()

            try:
                new_board = chess.Board(fen_string)
                if not new_board.is_valid():
                    raise ValueError("Generated FEN is invalid.")
            except ValueError as e:
                print(f"Error: {e}")
                return top_moves 

            board = new_board
            
            try:
                stockfish.set_fen_position(board.fen())
                top_moves = stockfish.get_top_moves(1)
            except Exception as e:
                print(f"Stockfish error: {e}")
            
            return top_moves
            
        board = chess.Board(None)
        board.turn = chess.WHITE
        top_moves = set_up_board(board, piece_positions, top_moves)
        
        img_with_Ans = img_with_grid.copy()
        for move in top_moves:
            start_label = move['Move'][:2]  
            end_label = move['Move'][2:]
            
            for i in range(rows):
                for j in range(cols):
                    x = i * cell_width
                    y = j * cell_height
                    label = columns[i] + rows_labels[j]
                    
                    text_x = x + cell_width // 2
                    text_y = y + cell_height // 2
                    
                    if label == start_label:
                        start_x = text_x
                        start_y = text_y
                        cv2.circle(img_with_Ans, (int(text_x), int(text_y)), 5, (0, 0, 255), -1)
                        cv2.putText(img_with_Ans, label, (text_x, text_y), cv2.FONT_HERSHEY_SIMPLEX, 1, (100, 0, 255), 2)
                    
                    if label == end_label:
                        end_x = text_x
                        end_y = text_y
                        cv2.circle(img_with_Ans, (int(text_x), int(text_y)), 5, (0, 0, 255), -1)
                        cv2.putText(img_with_Ans, label, (text_x, text_y), cv2.FONT_HERSHEY_SIMPLEX, 1, (100, 0, 255), 2)
            
            cv2.arrowedLine(img_with_Ans, (start_x, start_y), (end_x, end_y), (255, 0, 0), 1, tipLength=0.1)
        
        
        cv2.imshow("Chessboard", img_with_Ans)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

video.release()
cv2.destroyAllWindows()



0: 480x800 (no detections), 142.9ms
Speed: 4.0ms preprocess, 142.9ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 800)
No masks found in the results.

0: 480x800 (no detections), 134.6ms
Speed: 4.0ms preprocess, 134.6ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 800)
No masks found in the results.

0: 480x800 (no detections), 132.9ms
Speed: 3.0ms preprocess, 132.9ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 800)
No masks found in the results.

0: 480x800 (no detections), 139.0ms
Speed: 3.1ms preprocess, 139.0ms inference, 1.1ms postprocess per image at shape (1, 3, 480, 800)
No masks found in the results.

0: 480x800 (no detections), 129.9ms
Speed: 3.1ms preprocess, 129.9ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 800)
No masks found in the results.

0: 480x800 (no detections), 127.5ms
Speed: 2.0ms preprocess, 127.5ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 800)
No masks found in the results.

0: 

In [2]:

model_path = os.path.join('.', 'runs', 'segment', 'train_CB', 'weights', 'best.pt')
model = YOLO(model_path)

model_path1 = os.path.join('.', 'runs', 'detect', 'train_GCP', 'weights', 'best.pt')
model1 = YOLO(model_path1)

video_path = "C:/Users/Daithoulung Rongmai/Videos/Screen Recordings/Screen Recording 2024-10-25 212252.mp4" #Enter the path to the video file
video = cv2.VideoCapture(video_path)

if not video.isOpened():
    print("Error: Could not open video stream.")
    exit()

stockfish=Stockfish("StockFish/stockfish-windows-x86-64-avx2.exe")

top_moves = []

while True:
    ret, frame = video.read()
    if not ret:
        print("Error: Failed to capture frame.")
        break

    results = model.predict(frame, conf=0.5)

    warping_done = False

    for result in results:
        
        if hasattr(result, 'masks') and result.masks is not None:
            masks = result.masks.data.cpu().numpy()  

            for i in range(masks.shape[0]):
                
                mask = masks[i]
                mask_binary = (mask * 255).astype(np.uint8)
                mask_binary_resized = cv2.resize(mask_binary, (frame.shape[1], frame.shape[0]), interpolation=cv2.INTER_NEAREST)
                masked_frame = cv2.bitwise_and(frame, frame, mask=mask_binary_resized)
                contours, _ = cv2.findContours(mask_binary_resized, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
                
                if contours:
                    
                    largest_contour = max(contours, key=cv2.contourArea)
                    epsilon = 0.02 * cv2.arcLength(largest_contour, True)
                    approx_corners = cv2.approxPolyDP(largest_contour, epsilon, True)

                    if len(approx_corners) == 4:
                        
                        src_pts = np.array([pt[0] for pt in approx_corners], dtype="float32")
                        dst_pts = np.array([[0, 0], [600, 0], [600, 600], [0, 600]], dtype="float32")
                        
                        M = cv2.getPerspectiveTransform(src_pts, dst_pts)
                        warped_frame = cv2.warpPerspective(masked_frame, M, (600, 600))
                        rotated_frame = cv2.rotate(warped_frame, cv2.ROTATE_90_CLOCKWISE)
                        mirrored_frame = cv2.flip(rotated_frame, 1)

                        grid_size = 600
                        rows, cols = 8, 8
                        cell_width = grid_size // cols
                        cell_height = grid_size // rows

                        img_with_grid = mirrored_frame.copy()
                        for r in range(1, rows):
                            y = r * cell_height
                            cv2.line(img_with_grid, (0, y), (grid_size, y), (255, 0, 0), 1)
                        for c in range(1, cols):
                            x = c * cell_width
                            cv2.line(img_with_grid, (x, 0), (x, grid_size), (255, 0, 0), 1)

                        warping_done = True
                    else:
                        print("Could not find 4 corners for perspective transformation.")
                else:
                    print("No contours found for the mask.")
        else:
            print("No masks found in the results.")
    
    if not warping_done:
        for result in results:
            if hasattr(result, 'boxes') and result.boxes is not None:
                boxes = result.boxes.xyxy.cpu().numpy() 
                
                if len(boxes) > 0:
                    print("in")
                    x_min, y_min, x_max, y_max = boxes[0]
                    src_pts = np.array([[x_min, y_min], [x_max, y_min], [x_max, y_max], [x_min, y_max]], dtype="float32")
                    dst_pts = np.array([[0, 0], [600, 0], [600, 600], [0, 600]], dtype="float32")

                    M = cv2.getPerspectiveTransform(src_pts, dst_pts)
                    warped_frame = cv2.warpPerspective(frame, M, (600, 600))
                    rotated_frame = cv2.rotate(warped_frame, cv2.ROTATE_90_CLOCKWISE)
                    mirrored_frame = cv2.flip(rotated_frame, 1)

                    grid_size = 600
                    rows, cols = 8, 8
                    cell_width = grid_size // cols
                    cell_height = grid_size // rows
                    
                    img_with_grid = mirrored_frame.copy()
                    print(img_with_grid)
                    for r in range(1, rows):
                        y = r * cell_height
                        cv2.line(img_with_grid, (0, y), (grid_size, y), (255, 0, 0), 1)
                    for c in range(1, cols):
                        x = c * cell_width
                        cv2.line(img_with_grid, (x, 0), (x, grid_size), (255, 0, 0), 1)
                        
                    warping_done = True
                    break

    if not warping_done:
        frameResized = cv2.resize(frame, (600, 600))
        cv2.imshow("Chessboard", frameResized)
            
    else:
        columns = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h']
        rows_labels = ['1', '2', '3', '4', '5', '6', '7', '8']
        rows_labels.reverse()
    
        img_with_labels = img_with_grid.copy()
    
        for i in range(rows):
            for j in range(cols):
                x = i * cell_width
                y = j * cell_height
        
                label = columns[i] + rows_labels[j]
    
                text_x = x + cell_width // 2 - 15 
                text_y = y + cell_height // 2 + 15
        
                cv2.putText(img_with_labels, label, (text_x, text_y), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
    
        res = img_with_labels.copy()
    
        piece_positions = {}
        
        results1 = model1.predict(mirrored_frame, conf=0.7)
        
        for result in results1:
            
            boxes = result.boxes.xyxy.cpu().numpy()
            classes = result.boxes.cls.cpu().numpy()
            names = result.names
            
            for box,class_id in zip(boxes,classes):
                
                x_min,y_min,x_max,y_max = box
                center_x = (x_min + x_max) / 2
                center_y = ((y_min + y_max) / 2)
                
                grid_col = int(center_x // cell_width)
                grid_row = int(center_y // cell_height)
                
                if 0 <= grid_col < cols and 0 <= grid_row < rows:
                    
                    grid_label = columns[grid_col] + rows_labels[grid_row]
                    piece_positions[grid_label] = names[int(class_id)]
                    
                    cv2.circle(res, (int(center_x), int(center_y)), 5, (255, 0, 0), -1)
                    cv2.putText(res, grid_label, (int(center_x)-10, int(center_y)-15), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
    
        piece_map = {
            'white-rook': chess.Piece(chess.ROOK, chess.WHITE),
            'white-knight': chess.Piece(chess.KNIGHT, chess.WHITE),
            'white-bishop': chess.Piece(chess.BISHOP, chess.WHITE),
            'white-queen': chess.Piece(chess.QUEEN, chess.WHITE),
            'white-king': chess.Piece(chess.KING, chess.WHITE),
            'white-pawn': chess.Piece(chess.PAWN, chess.WHITE),
            'black-rook': chess.Piece(chess.ROOK, chess.BLACK),
            'black-knight': chess.Piece(chess.KNIGHT, chess.BLACK),
            'black-bishop': chess.Piece(chess.BISHOP, chess.BLACK),
            'black-queen': chess.Piece(chess.QUEEN, chess.BLACK),
            'black-king': chess.Piece(chess.KING, chess.BLACK),
            'black-pawn': chess.Piece(chess.PAWN, chess.BLACK),
        }
        
        def set_up_board(board, piece_positions, top_moves):
    
            try:
                for position, piece_description in piece_positions.items():
                    piece = piece_map.get(piece_description)
                    if piece:
                        square = chess.parse_square(position)
                        board.set_piece_at(square, piece)
                    else:
                        raise ValueError(f"Invalid piece description: {piece_description}")
            except Exception as e:
                print(f"Error setting up board: {e}")
                return top_moves
        
            fen_string = board.fen()

            try:
                new_board = chess.Board(fen_string)
                if not new_board.is_valid():
                    raise ValueError("Generated FEN is invalid.")
            except ValueError as e:
                print(f"Error: {e}")
                return top_moves 

            board = new_board
            
            try:
                stockfish.set_fen_position(board.fen())
                top_moves = stockfish.get_top_moves(1)
            except Exception as e:
                print(f"Stockfish error: {e}")
            
            return top_moves
            
        board = chess.Board(None)
        board.turn = chess.WHITE
        top_moves = set_up_board(board, piece_positions, top_moves)
        
        img_with_Ans = img_with_grid.copy()
        for move in top_moves:
            start_label = move['Move'][:2]  
            end_label = move['Move'][2:]
            
            for i in range(rows):
                for j in range(cols):
                    x = i * cell_width
                    y = j * cell_height
                    label = columns[i] + rows_labels[j]
                    
                    text_x = x + cell_width // 2
                    text_y = y + cell_height // 2
                    
                    if label == start_label:
                        start_x = text_x
                        start_y = text_y
                        cv2.circle(img_with_Ans, (int(text_x), int(text_y)), 5, (0, 0, 255), -1)
                        cv2.putText(img_with_Ans, label, (text_x, text_y), cv2.FONT_HERSHEY_SIMPLEX, 1, (100, 0, 255), 2)
                    
                    if label == end_label:
                        end_x = text_x
                        end_y = text_y
                        cv2.circle(img_with_Ans, (int(text_x), int(text_y)), 5, (0, 0, 255), -1)
                        cv2.putText(img_with_Ans, label, (text_x, text_y), cv2.FONT_HERSHEY_SIMPLEX, 1, (100, 0, 255), 2)
            
            cv2.arrowedLine(img_with_Ans, (start_x, start_y), (end_x, end_y), (255, 0, 0), 1, tipLength=0.1)
        
        
        cv2.imshow("Chessboard", img_with_Ans)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

video.release()
cv2.destroyAllWindows()



0: 800x800 1 0, 370.7ms
Speed: 8.0ms preprocess, 370.7ms inference, 5.0ms postprocess per image at shape (1, 3, 800, 800)

0: 800x800 2 black-bishops, 1 black-king, 5 black-pawns, 1 black-rook, 1 white-king, 4 white-pawns, 1 white-queen, 1 white-rook, 224.8ms
Speed: 8.5ms preprocess, 224.8ms inference, 1.0ms postprocess per image at shape (1, 3, 800, 800)

0: 800x800 1 0, 321.6ms
Speed: 7.0ms preprocess, 321.6ms inference, 3.0ms postprocess per image at shape (1, 3, 800, 800)

0: 800x800 2 black-bishops, 1 black-king, 5 black-pawns, 1 black-rook, 1 white-king, 4 white-pawns, 1 white-queen, 1 white-rook, 225.1ms
Speed: 8.0ms preprocess, 225.1ms inference, 0.0ms postprocess per image at shape (1, 3, 800, 800)

0: 800x800 1 0, 296.8ms
Speed: 8.0ms preprocess, 296.8ms inference, 2.0ms postprocess per image at shape (1, 3, 800, 800)

0: 800x800 2 black-bishops, 1 black-king, 5 black-pawns, 1 black-rook, 1 white-king, 4 white-pawns, 1 white-queen, 1 white-rook, 218.4ms
Speed: 8.0ms preproce